In [1]:
import torch
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
device

'cuda:0'

In [2]:
import torch.nn as nn
import numpy as np
from scipy.stats import ortho_group

In [3]:
class ActorNetwork(nn.Module):
    def __init__(self, feature_extractor_network: nn.Module, feature_dims : int, parameter_vector_dims : int):
        super().__init__()
        self.feature_extractor_network = feature_extractor_network
        
        self.actor_head = nn.Sequential(
            # Note:
            # input num dims = 2 * feature_dims + parameter_vector_dims
            # because actor head will get a concatenation of the following:
            #    - feature vector of target spectrogram
            #    - feature vector of prior spectrogram generated from the prior predicted parameter vector
            #    - feature vector of prior predicted parameter vector
            nn.Linear(2 * feature_dims + parameter_vector_dims, parameter_vector_dims),
            nn.ReLU(),
            nn.Linear(parameter_vector_dims, parameter_vector_dims),

            # This final sigmoid activation ensures each param output is in the [0,1] range
            nn.Sigmoid()
        )

    def forward(self, target_spectrogram, prior_spectrogram, prior_parameter_vector):
        # Extract spectrogram features
        target_spectrogram_feats = self.feature_extractor_network(target_spectrogram)
        prior_spectrogram_feats = self.feature_extractor_network(prior_spectrogram)
        state_vec = torch.hstack((target_spectrogram_feats, prior_spectrogram_feats, prior_parameter_vector))
        
        # Get action from actor network
        pi = self.actor_head(state_vec)
        
        return pi

In [4]:
class CriticNetwork(nn.Module):
    def __init__(self, feature_extractor_network: nn.Module, feature_dims : int, parameter_vector_dims : int):
        super().__init__()
        self.feature_extractor_network = feature_extractor_network
                
        self.critic_head = nn.Sequential(
            # See prior note for explanation behind number of input dims here
            nn.Linear(2 * feature_dims + parameter_vector_dims, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            
            # We use a final tanh for the critic head's output value, to squash it between 1 and -1
            # nn.Tanh()
        )
        
    def forward(self, target_spectrogram, prior_spectrogram, prior_parameter_vector):
        # Extract spectrogram features
        target_spectrogram_feats = self.feature_extractor_network(target_spectrogram)
        prior_spectrogram_feats = self.feature_extractor_network(prior_spectrogram)
        state_vec = torch.hstack((target_spectrogram_feats, prior_spectrogram_feats, prior_parameter_vector))
        
        # Get predicted Q(s,a) value from critic network
        value = self.critic_head(state_vec)
        
        return value

In [5]:
spectrogram_shape = (1, 4, 8)
parameter_vector_dims = 16 #np.prod(spectrogram_shape)
feature_vector_dims=32

In [6]:
# [parameter vector dims, spectrogram flattened num dims]
# parameter_vector_to_spectrogram_matrix = ortho_group.rvs(dim=np.prod(spectrogram_shape)).astype(np.float32)
# parameter_vector_to_spectrogram_matrix = torch.from_numpy(parameter_vector_to_spectrogram_matrix).to(device).requires_grad_(False) 
parameter_vector_to_spectrogram_matrix = torch.randn(parameter_vector_dims, np.prod(spectrogram_shape), requires_grad=False, device=device)
parameter_vector_to_spectrogram_matrix.shape

torch.Size([16, 32])

In [7]:
def generate_spectrogram_from_parameter_vector(parameter_vector):
    # [N, spectrogram flattened num dims]
    spectrogram_flattened = parameter_vector @ parameter_vector_to_spectrogram_matrix
    spectrogram = torch.reshape(spectrogram_flattened, (-1,) + spectrogram_shape)
    return spectrogram

## Demo training

In [8]:
from torch.optim import Adam
from tqdm.auto import tqdm

In [9]:
ground_truth_params_vector = torch.randn(1, parameter_vector_dims, device=device)
ground_truth_spectrogram = generate_spectrogram_from_parameter_vector(ground_truth_params_vector)
ground_truth_params_vector.shape, ground_truth_spectrogram.shape

(torch.Size([1, 16]), torch.Size([1, 1, 4, 8]))

In [10]:
# These should be close if parameter_vector_to_spectrogram_matrix is orthonormal
# ground_truth_params_vector, (ground_truth_spectrogram.reshape(1, -1) @ parameter_vector_to_spectrogram_matrix.T)

In [11]:
def compute_mse_between_spectrograms(spec1, spec2):
    # output: [N,1]
    return ((spec1 - spec2)**2).mean([1,2,3]).unsqueeze(-1)

In [12]:
def compute_reward(spec1, spec2):
    # output: [N,1]
    
    # first, we compute mse loss and use that to construct a reward
    mse_loss = compute_mse_between_spectrograms(spec1, spec2)
    
    # with the transformation below, higher mse loss -> lower reward, and lower mse loss -> higher reward
    reward = -mse_loss #1/mse_loss #-torch.log(mse_loss) #1/mse_loss
    
    # finally, to keep reward within [-1,1] range, we use tanh
    # reward = torch.tanh(reward)
    
    return reward

In [13]:
def compute_returns(rewards, gamma=0.99):
    """
    https://github.com/yc930401/Actor-Critic-pytorch/blob/5d359bc93839357255b591c0a87fc42542854eb8/Actor-Critic.py#L50
    """
    returns = []
    
    R = rewards[-1]

    for idx, step in enumerate(reversed(range(len(rewards)))):
        if idx == 0:
            R = rewards[step]
        else:
            R = rewards[step] + gamma * R
        returns.insert(0, R)
    return returns

In [14]:
class DummySpectrogramFeatureExtractorNetwork(nn.Module):
    """
    This is a dummy placeholder spectrogram feature extractor, that will convert any input spectrogram into a output feature vector.
    The pre-trained VAE, for example, would ideally replace this.
    """
    
    def __init__(self, spectrogram_shape : tuple[int], feature_dims : int):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(spectrogram_shape), feature_dims),
            nn.ReLU(),
            nn.Linear(feature_dims, feature_dims)
        )
    
    def forward(self, spectrogram):
        # flatten spectrogram into [N, flattened dims]
        spectrogram_flattened = torch.flatten(spectrogram, start_dim=1)
        return self.model(spectrogram_flattened)

In [15]:
def get_next_state(predicted_parameter_vector):
    next_spectrogram = generate_spectrogram_from_parameter_vector(predicted_parameter_vector)
    return [ground_truth_spectrogram, next_spectrogram, predicted_parameter_vector]

In [16]:
ACTOR_LR = 1e-7
CRITIC_LR = 1e-6

num_episodes = 1_000_000
num_steps_per_episode = 1_000
gamma = 0.99

GRADIENT_CLIPPING = 0.5

step_optimizer_every_n_episode_steps = 32

In [17]:
feature_extractor_network = DummySpectrogramFeatureExtractorNetwork(spectrogram_shape=spectrogram_shape, feature_dims=feature_vector_dims)
_ = feature_extractor_network.to(device)

In [18]:
actor_model = ActorNetwork(feature_extractor_network=feature_extractor_network, feature_dims=feature_vector_dims, parameter_vector_dims=parameter_vector_dims).to(device)
critic_model = CriticNetwork(feature_extractor_network=feature_extractor_network, feature_dims=feature_vector_dims, parameter_vector_dims=parameter_vector_dims).to(device)

In [19]:
actor_optimizer = Adam(actor_model.parameters(), lr=ACTOR_LR)
critic_optimizer = Adam(critic_model.parameters(), lr=CRITIC_LR)

In [ ]:
episode_pbar = tqdm(range(num_episodes))
for episode_idx in episode_pbar: #range(num_episodes):
    # Set initial state for episode
    prior_predicted_parameter_vector = torch.zeros((1,parameter_vector_dims), device=device)
    prior_predicted_spectrogram = generate_spectrogram_from_parameter_vector(prior_predicted_parameter_vector)
    
    state = (ground_truth_spectrogram, prior_predicted_spectrogram, prior_predicted_parameter_vector)
    
    all_rewards = []
    
    episode_step_pbar = tqdm(range(num_steps_per_episode))
    for step_idx in episode_step_pbar:
        actor_optimizer.zero_grad()
        critic_optimizer.zero_grad()
        
        # Get predicted action and value for current state
        predicted_parameter_vector, value_of_current_state = actor_model(*state), critic_model(*state)
        
        # Perform action to get next state and reward
        next_state = get_next_state(predicted_parameter_vector)
        reward = compute_reward(ground_truth_spectrogram, next_state[1])
        
        value_next_state = critic_model(*next_state)
        
        #TD Target: r + gamma * V(next_state)
        td_target = reward + gamma * value_next_state
        
        delta = td_target - value_of_current_state
        
        actor_loss = delta * -torch.log(predicted_parameter_vector).mean(-1)
        critic_loss = delta ** 2
        
        # We're weighting critic loss by 0.5
        critic_loss_weight = 0.5
        loss = actor_loss + critic_loss_weight * critic_loss
        loss.backward()
        
        if step_idx % step_optimizer_every_n_episode_steps == 0:
            if GRADIENT_CLIPPING is not None:
                torch.nn.utils.clip_grad_norm_(actor_model.parameters(), GRADIENT_CLIPPING)
                torch.nn.utils.clip_grad_norm_(critic_model.parameters(), GRADIENT_CLIPPING)

            actor_optimizer.step()
            critic_optimizer.step()
        
        # Now that we've learned from this step, update current state variable
        next_state[1] = next_state[1].detach()
        next_state[2] = next_state[2].detach()
        state = next_state
        
        all_rewards.append(reward.item())
        
        with torch.no_grad():
            params_mse = ((predicted_parameter_vector - ground_truth_params_vector)**2).mean().item()
        
        episode_step_pbar.set_postfix({"loss" : loss.item(), "reward" : reward.item(), "params_mse" : params_mse})
    
    # print (f"Episode {episode_idx + 1}: total reward: {sum(all_rewards)}, return: {compute_returns(all_rewards, gamma)[0]}")
    episode_pbar.set_postfix({"total reward" : sum(all_rewards), "return" : compute_returns(all_rewards, gamma)[0]})

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]